In [38]:
import os
import csv
import json
import logging
import unicodedata
import pandas as pd
from time import time
from datetime import datetime
from elasticsearch import Elasticsearch, ElasticsearchException, helpers

In [16]:
# ==================================================================== #
# config variables
# ==================================================================== #

with open('config-copy.json') as config_file:
    config = json.load(config_file)

loggername = config['loggername']
csv_files = config['csv_file']
csv_file_delimiter = config['csv_file_delimiter']
csv_reader_encoding = config['csv_reader_encoding']
elastic_hosts = config['elastic_hosts']
es_index = config['es_index']
es_doc_type = config['es_doc_type']
es_id_key = config['es_id_key']
category_column = config['category_column']
mapping = config['mapping']
id_column = config['id_column']
outter_key = config['outter_key']
dump = config['dump']

In [75]:
# ==================================================================== #
# functions
# ==================================================================== #
def load_csv(filepath: str, delimiter: str, header='infer', encoding='utf-8'):
    print('Loading {}'.format(filepath))
    return pd.read_csv(filepath, delimiter=delimiter, header=header, encoding=encoding, dtype=object)

def find_ids(df, id_column: str):
    return list(set(df[id_column]))

def id_count(df, id_column: str):
    return len(list(set(df[id_column])))

def single_id_df(df, id_column: str, id_value: any):
    return df[df[id_column] == id_value]

def remove_accents(input_str):
    nkfd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nkfd_form if not unicodedata.combining(c)])

def extract_features_by_category(single_id_df, category: str, related_features: list, category_column: str):
    lc = [ [remove_accents(str(reg[i]).rstrip().lower()) for i in range(len(reg)) ] 
          for reg in single_id_df[single_id_df[category_column] == category][related_features].values ]
    related_features = [feat.lower() for feat in related_features]
    out = []
    for reg in lc:
        out.append(dict(zip(related_features, reg)))
    category_key = category.lower()
    category_dict = {category_key: out}
    return category_dict

def elastic_bulk_index (index: str, docType: str, data: list, elastic, _id_key: str):
    bulk = [{"_index": index, "_type": docType, "_id": reg[_id_key], "_source": reg} for reg in data]
    return helpers.bulk(client=elastic, actions=bulk)[0]

def sentRate(total: int, good: int):
    acc = 100.0 - ((total - good)/total)
    print('Delivery rate {}%'.format(acc))
    return acc

def dump_json(obj: dict, yes_or_no: str):
    if yes_or_no.lower() == "yes":
        file_name = 'dump_{}.json'.format(datetime.now()).replace(" ", "_")
        with open('dump/{}'.format(file_name), 'w') as json_file:
            json.dump(obj, json_file)
            print('Generated {}'.format(file_name))
    return yes_or_no

In [85]:
def curriculum_json_generator(df, field_map: dict, id_column: str, outter_key: str, category_column: str):
    id_list = find_ids(df=df, id_column=id_column)
    print('Found {} units on \'{}\' to process'.format(len(id_list), id_column))
    out = []
    for f_id in id_list:
        f_info = {'matricula': str(f_id), '@timestamp': datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"), outter_key: {} }
        f_df = single_id_df(df=df, id_column=id_column, id_value=f_id)
        for key in field_map.keys():
            try:
                data = extract_features_by_category(single_id_df=f_df, category=key, category_column=category_column,
                                                    related_features=field_map[key])[key.lower()]
                f_info[outter_key][key.lower()] = data
            except:
                print('id: {} key: \'{}\''.format(f_id, key))
        print(f_info[outter_key]['apresentacao'])
        out.append(f_info)
    print('Generated: {}. Delta: {}'.format(len(out), len(out)-len(id_list)))
    return out

In [88]:
for csv_file in csv_files[:1]:
    df = load_csv(filepath=csv_file, delimiter=csv_file_delimiter, header='infer', encoding=csv_reader_encoding)
    obj = curriculum_json_generator(df=df, field_map=mapping, id_column=id_column, outter_key=outter_key, category_column=category_column)
    # bulk = elastic_bulk_index(index=es_index, docType=es_doc_type, data=obj, _id_key=es_id_key, elastic=es)
    # sr = sentRate(total=len(obj), good=bulk)
    # dump_json(obj=obj, yes_or_no=dump)

Loading /home/f4119597/Downloads/Arquivos_Finais/201910031133__0000000_A_1999999.csv
Found 645 units on 'MATRICULA_FUNCIONARIO' to process
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[{'indicador_apresentacao': 'nan', 'apresentacao': 'meu trabalho tem como objetivo colaborar para que o banco do brasil cumpra os objetivos e metas estabelecidas. primo pela qualidade dos servicos prestados, com etica, comprometimento e flexibilidade, buscando a satisfacao dos clientes internos e externos. atuo com planejamento e motivacao para gerir recursos, processos e informacoes, observando sempre o direcionamento da empresa, para que continue sendo um banco confiavel, competitivo e participativo no processo de desenvolvimento do brasil.'}]
[]
[]
[]
[]
[]
[]
[{'indicador_apresentacao': 'nan', 'apresentacao': 'sou administrador e analistas de sistemas, com mba em governanca de ti com enfase em ambiente mainframe. sou interessado por tecnologia, desenvolvimento e metodos ageis, e sempre estou 

[]
[{'indicador_apresentacao': 'nan', 'apresentacao': 'vaca nao da leite. muitas pessoas falam que eu aprendo as coisas com muita facilidade. sabemos que nao e bem assim. meu primeiro codigo executavel fiz aos 11 anos de idade em 1989 usando um microcomputador cp500 da prologica e a linguagem de programacao basic. de la pra ca foram muitos empregos, estudos e muita leitura. fruto de muito esforco e nao e facil. o trabalho e coletivo mas comeca com o esforco individual. e o leite? se nao tivermos atitude, conhecimento e a habilidade necessaria ele nao vira.'}]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[{'indicador_apresentacao': 'nan', 'apresentacao': 'tenho 34 anos, sou brasiliense, casado e tenho 2 filhas: ana mel de 13 anos e heloisa de 2 anos. tenho 8 anos de banco, 2 anos de ditec. sou graduado em sistemas de informacao, pela facnet (atualmente faculdade anhanguera) e pos graduado em gestao de ti e big data analytics. atualmente, faco parte do time de intercambio de 